# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install geoviews

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,erdenet,49.0333,104.0833,-0.63,94,20,1.17,MN,1696279473
1,1,port alfred,-33.5906,26.8910,16.77,91,0,0.45,ZA,1696279473
2,2,margate,51.3813,1.3862,19.06,94,100,0.89,GB,1696279473
3,3,avarua,-21.2078,-159.7750,22.03,73,75,4.12,CK,1696279474
4,4,pangody,65.8500,74.4859,9.12,89,100,8.37,RU,1696279474


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Dependencies and Setup
import geoviews as gv
import geoviews.tile_sources as gts

import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Define the Geoapify API endpoint for geocoding
geoapify_endpoint = f"https://api.geoapify.com/v2/geocoding/search?apiKey={geoapify_key}"


# Display sample data
city_data_df.head()

# Create the map plot using hvplot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',           # Longitude
    y='Lat',           # Latitude
    c='Humidity',      # Color by humidity
    cmap='viridis',    # Colormap for color representation
    size='Humidity',   # Size by humidity
    title='City Humidity',  # Plot title
    hover_cols=['City', 'Humidity', 'Country'],  # Data to show on hover
    frame_height=400,  # Height of the plot
    frame_width=600,   # Width of the plot
)

# Display the map
map_plot

C:\Users\mklut\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
city_data_df.head(6)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,erdenet,49.0333,104.0833,-0.63,94,20,1.17,MN,1696279473
1,1,port alfred,-33.5906,26.8910,16.77,91,0,0.45,ZA,1696279473
2,2,margate,51.3813,1.3862,19.06,94,100,0.89,GB,1696279473
3,3,avarua,-21.2078,-159.7750,22.03,73,75,4.12,CK,1696279474
4,4,pangody,65.8500,74.4859,9.12,89,100,8.37,RU,1696279474
5,5,waitangi,-43.9535,-176.5597,11.57,88,95,0.89,NZ,1696279474


In [7]:
# Display summary statistics for columns of interest
summary_stats = city_data_df[['Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']].describe()
print(summary_stats)


         Max Temp    Humidity  Cloudiness  Wind Speed
count  572.000000  572.000000  572.000000  572.000000
mean    19.489703   70.286713   54.821678    3.721888
std      9.439292   20.553845   40.223452    2.538013
min     -8.360000    8.000000    0.000000    0.000000
25%     12.957500   60.750000    9.750000    1.790000
50%     21.220000   76.000000   66.000000    3.090000
75%     26.650000   86.000000   99.000000    5.075000
max     40.200000  100.000000  100.000000   15.240000


In [8]:
# Relax the criteria to find cities with conditions closer to the example
# Filter cities with Max Temp between 293K and 303K (approximately 20°C to 30°C), Humidity below 85%, Cloudiness equal to or below 10%, and Wind Speed below 5 m/s
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= 293) & (city_data_df['Max Temp'] <= 303) &
                                   (city_data_df['Humidity'] < 85) & (city_data_df['Cloudiness'] <= 10) &
                                   (city_data_df['Wind Speed'] < 5)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head


<bound method NDFrame.head of Empty DataFrame
Columns: [City_ID, City, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed, Country, Date]
Index: []>

### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,erdenet,MN,49.0333,104.0833,94,
1,port alfred,ZA,-33.5906,26.8910,91,
2,margate,GB,51.3813,1.3862,94,
3,avarua,CK,-21.2078,-159.7750,73,
4,pangody,RU,65.8500,74.4859,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Import necessary libraries
import pandas as pd
import requests
from api_keys import geoapify_key

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Define the Geoapify API endpoint for hotel search
hotel_search_endpoint = "https://api.geoapify.com/v2/places"

# Define the parameters for the API request
params = {
    'categories': 'accommodation.hotel',
    'filter': f'circle:{city_data_df["Lng"].iloc[0]},{city_data_df["Lat"].iloc[0]},10000',  # Using the first city's coordinates
    'bias': f'proximity:{city_data_df["Lng"].iloc[0]},{city_data_df["Lat"].iloc[0]}',
    'limit': 20,  # Adjust the limit as needed
    'apiKey': geoapify_key
}

# Create a DataFrame called hotel_df to store hotel information
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

# Iterate through each row in the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Update the filter and bias parameters with the city's coordinates
    params['filter'] = f'circle:{row["Lng"]},{row["Lat"]},10000'
    params['bias'] = f'proximity:{row["Lng"]},{row["Lat"]}'

    # Make the API request
    response = requests.get(hotel_search_endpoint, params=params)
    data = response.json()

    # Check if 'features' key exists and if the first feature has a 'name' key
    if 'features' in data and data['features'] and 'name' in data['features'][0]['properties']:
        hotel_name = data['features'][0]['properties']['name']
    else:
        hotel_name = "No hotel found"

    # Update the "Hotel Name" column in the DataFrame with the hotel_name
    hotel_df.at[index, 'Hotel Name'] = hotel_name

    # Display the result
    print(f"{row['City']} - nearest hotel: {hotel_name}")



erdenet - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
margate - nearest hotel: The Reading Rooms
avarua - nearest hotel: Paradise Inn
pangody - nearest hotel: Газпром энерго
waitangi - nearest hotel: Hotel Chathams
ghadamis - nearest hotel: فندق عين الفارس
west island - nearest hotel: Cocos Village Bungalows
volosovo - nearest hotel: Наири
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
san antonio de pale - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
albany - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
sao joao da barra - nearest hotel: Pousada Mediterrâneo
santa maria - nearest hotel: Hotel Morotin
constantia - nearest hotel: Hotel Chérica
adamstown - nearest hotel: No hotel found
smidovich - nearest hotel: Восток
blackmans bay - nearest hotel: Villa Howden
vorgashor - nearest hotel: София
puerto natales - nearest hotel: Dorotea Patagonia Hostel
nabire - nearest 

In [13]:
# Display the updated DataFrame
hotel_df.head(5)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,erdenet,MN,49.0333,104.0833,94,No hotel found
1,port alfred,ZA,-33.5906,26.8910,91,No hotel found
2,margate,GB,51.3813,1.3862,94,The Reading Rooms
3,avarua,CK,-21.2078,-159.7750,73,Paradise Inn
4,pangody,RU,65.8500,74.4859,89,Газпром энерго


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
# Save hotel_df to a CSV file
hotel_df.to_csv('hotel_data.csv', index=False)
hotel_df = pd.read_csv("hotel_data.csv")

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np  # Import NumPy library

# Create the map plot using hvplot, including the 'Hotel Name' column
map_plot = hotel_df.hvplot.scatter(
    x='Lng',           # Longitude
    y='Lat',           # Latitude
    c='Humidity',      # Color by humidity
    cmap='viridis',    # Colormap for color representation
    size='Humidity',   # Size by humidity
    title='City Humidity with Hotel Names',  # Plot title
    hover_cols=['City', 'Humidity', 'Hotel Name'],  # Data to show on hover (including city name, humidity, and hotel name)
    frame_height=400,  # Height of the plot
    frame_width=600,   # Width of the plot
)

# Display the map
map_plot


:Scatter   [Lng]   (Lat,Humidity,City,Hotel Name)